##### Copyright 2018 The AdaNet Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Customizing AdaNet

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/adanet/blob/master/adanet/examples/tutorials/customizing_adanet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/adanet/blob/master/adanet/examples/tutorials/customizing_adanet.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


Often times, as a researcher or machine learning practitioner, you will have
some prior knowledge about a dataset. Ideally you should be able to encode that
knowledge into your machine learning algorithm. With `adanet`, you can do so by
defining the *neural architecture search space* that the AdaNet algorithm should
explore.

In this tutorial, we will explore the flexibility of the `adanet` framework, and
create a custom search space for an image-classification dataset using high-level
TensorFlow libraries like the
[`tf.keras.layers`](https://www.tensorflow.org/guide/keras#build_advanced_models)
functional API.



In [ ]:
#@test {"skip": true}
# If you're running this in Colab, first install the adanet package:
# !pip install adanet

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
import shutil
import numpy as np

import adanet
from adanet.examples import simple_dnn
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf
import tensorflow as tf

# The random seed to use.
RANDOM_SEED = 42

LOG_DIR = '/tmp/models'

## Download the data

Conveniently, the data is available via Keras:

In [3]:
(x_train, y_train), (x_test, y_test) = (
    tf.keras.datasets.boston_housing.load_data())

In [15]:
LOG_DIR = "/tmp/models"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Install ngrok binary.
# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip ngrok-stable-linux-amd64.zip

# Delete old logs dir.
shutil.rmtree(LOG_DIR, ignore_errors=True)

print("Follow this link to open TensorBoard in a new tab.")
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Follow this link to open TensorBoard in a new tab.
http://f0fa9d950d3b.ngrok.io


## Supply the data in TensorFlow

Our first task is to supply the data in TensorFlow. Using the
tf.estimator.Estimator covention, we will define a function that returns an
`input_fn` which returns feature and label `Tensors`.

We will also use the `tf.data.Dataset` API to feed the data into our models.

In [16]:
feat_nam_lst = ['CRIM','ZN','INDUS','CHAS','NOX','RM', 'AGE','DIS', 'RAD','TAX','PTRATIO','B', 'LSTAT']

def input_fn(partition):
    def _input_fn():
        feat_tensor_dict = {}
        if partition == 'train':
            x = x_train.copy()
            y = y_train.copy()
        else:
            x = x_test.copy()
            y = y_test.copy()
        for i in range(0, np.size(x, 1)):
            feat_nam = feat_nam_lst[i]
            feat_tensor_dict[feat_nam] = tf.convert_to_tensor(
                tf.math.log1p(x[:, i]), dtype=tf.float64)
            label_tensor = tf.convert_to_tensor(tf.math.log1p(y), dtype=tf.float64)
        return (feat_tensor_dict, label_tensor)

    return _input_fn

# def input_fn(partition, training, batch_size):
#     """Generate an input function for the Estimator."""

#     def _input_fn():

#         if partition == "train":
#           dataset = tf.data.Dataset.from_tensor_slices(({
#               FEATURES_KEY: tf.log1p(x_train)
#           }, tf.log1p(y_train)))
#         else:
#           dataset = tf.data.Dataset.from_tensor_slices(({
#               FEATURES_KEY: tf.log1p(x_test)
#           }, tf.log1p(y_test)))

#         # We call repeat after shuffling, rather than before, to prevent separate
#         # epochs from blending together.
#         if training:
#           dataset = dataset.shuffle(10 * batch_size, seed=RANDOM_SEED).repeat()

#         dataset = dataset.batch(batch_size)
#         iterator = dataset.make_one_shot_iterator()
#         features, labels = iterator.get_next()
#         return features, labels

#     return _input_fn

#['feat' + str(i) for i in range(0, np.size(x_train, 1))]

feature_columns = []
for item in feat_nam_lst:
    feature_columns.append(tf.feature_column.numeric_column(item))

head = tf.estimator.RegressionHead(1)

LEARNING_RATE = 0.001  #@param {type:"number"}
TRAIN_STEPS = 60000  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}

LEARN_MIXTURE_WEIGHTS = False  #@param {type:"boolean"}
ADANET_LAMBDA = 0  #@param {type:"number"}
ADANET_ITERATIONS = 3  #@param {type:"integer"}

max_iteration_steps = TRAIN_STEPS // ADANET_ITERATIONS

config = tf.estimator.RunConfig(model_dir=LOG_DIR)
estimator = adanet.AutoEnsembleEstimator(
  head=head,
  candidate_pool=lambda config: {
#       'lr': tf.estimator.LinearEstimator(head=head, feature_columns=feature_columns),
      'dnn1': tf.estimator.DNNRegressor(feature_columns=feature_columns, hidden_units=[10, 50]),
      'dnn2': tf.estimator.DNNRegressor(feature_columns=feature_columns, hidden_units=[10, 50, 100]),
#       'dnn3': tf.estimator.DNNRegressor(feature_columns=feature_columns, hidden_units=[20, 40, 80])
  },
#   ensemble_strategies=[adanet.ensemble.AllStrategy()],
  max_iteration_steps=max_iteration_steps,
  config=config,
  export_subnetwork_last_layer=False)

estimator.train(input_fn=input_fn(partition='train'), steps=TRAIN_STEPS)
metrics = estimator.evaluate(input_fn=input_fn(partition='test'), steps=1000)

metrics

INFO:tensorflow:Using config: {'_model_dir': '/tmp/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Saving dict for global step 60000: average_loss = 0.056238536, best_ensemble_index_0 = 1, best_ensemble_index_1 = 2, best_ensemble_index_2 = -1, global_step = 60000, iteration = 2, label/mean = 3.1049607, loss = 0.0562392, prediction/mean = 3.092037


INFO:tensorflow:Saving dict for global step 60000: average_loss = 0.056238536, best_ensemble_index_0 = 1, best_ensemble_index_1 = 2, best_ensemble_index_2 = -1, global_step = 60000, iteration = 2, label/mean = 3.1049607, loss = 0.0562392, prediction/mean = 3.092037


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60000: /tmp/models/model.ckpt-60000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 60000: /tmp/models/model.ckpt-60000


{'architecture/adanet/ensembles': b'\n6\n\x13architecture/adanetB\x15\x08\x07\x12\x00B\x0f| dnn2 | dnn2 |J\x08\n\x06\n\x04text',
 'average_loss': 0.056238536,
 'best_ensemble_index_0': 1,
 'best_ensemble_index_1': 2,
 'best_ensemble_index_2': -1,
 'iteration': 2,
 'label/mean': 3.1049607,
 'loss': 0.0562392,
 'prediction/mean': 3.092037,
 'global_step': 60000}

In [17]:
def ensemble_architecture(result):
    """Extracts the ensemble architecture from evaluation results."""

    architecture = result["architecture/adanet/ensembles"]
    # The architecture is a serialized Summary proto for TensorBoard.
    summary_proto = tf.compat.v1.summary.Summary.FromString(architecture)
    return summary_proto.value[0].tensor.string_val[0]

metrics

{'architecture/adanet/ensembles': b'\n6\n\x13architecture/adanetB\x15\x08\x07\x12\x00B\x0f| dnn2 | dnn2 |J\x08\n\x06\n\x04text',
 'average_loss': 0.056238536,
 'best_ensemble_index_0': 1,
 'best_ensemble_index_1': 2,
 'best_ensemble_index_2': -1,
 'iteration': 2,
 'label/mean': 3.1049607,
 'loss': 0.0562392,
 'prediction/mean': 3.092037,
 'global_step': 60000}

## Generating predictions on our trained model

Now that we've got a trained model, we can use it to generate predictions on new input. To keep things simple, here we'll generate predictions on our `estimator` using the first 10 examples from the test set.

In [20]:
LABEL='target'
import numpy as np

def get_input_fn(num_epochs=None, n_batch = 128, shuffle=False): 
    
    target = df_test.pop(LABEL)
    dataset = tf.data.Dataset.from_tensor_slices((df_test.to_dict('list'), target.values))
    dataset = dataset.batch(n_batch)
#     dataset = dataset.repeat(num_epochs)
    return dataset

# from sklearn.datasets import load_boston
import pandas as pd
# boston = load_boston()
# features = np.array(pd.Series(boston['feature_names'], dtype=pd.StringDtype()))
# features = np.concatenate([x_test, y_test]) 

# df_test =  pd.DataFrame(data= np.c_[x_test, y_test],
#                      columns= feat_nam_lst + ['target'] )
# y_pred = estimator.predict(input_fn=get_input_fn)

# y = df_test['target'].values
# y_pred = list(y_pred)

pred = np.array([arr['predictions'][0] for arr in y_pred])
from sklearn.metrics import r2_score
print(pred.shape)
r2 = r2_score(tf.math.log1p(y), tf.math.log1p(pred))
print(tf.math.log1p(y))
print(tf.math.log1p(pred))
print(r2)

(102,)
tf.Tensor(
[2.10413415 2.98568194 2.99573227 3.33220451 3.14415228 3.23867845
 3.47196645 3.17387846 3.06805294 3.18635263 2.97552957 2.74084002
 2.93385687 3.93182563 3.08190997 3.2308044  3.22684399 3.03495299
 3.00071982 3.16547505 2.56494936 2.41591378 3.04452244 2.97041447
 3.08648664 3.17805383 3.34990409 3.43720782 2.35137526 3.13549422
 3.10009229 2.71469474 3.52929738 3.19458313 3.04927304 2.12823171
 2.79728133 3.21084365 3.04927304 3.23867845 3.52636052 3.38099467
 2.71469474 3.8649314  3.51154544 3.42100001 3.38099467 3.03495299
 3.05400118 3.25809654 3.59456877 3.05870707 2.37024374 2.74084002
 3.5807373  3.31781577 2.10413415 3.93182563 3.5085559  3.11794991
 3.42751469 2.6461748  3.34990409 3.10009229 3.18221184 3.13113691
 2.63905733 3.18635263 2.20827441 1.88706965 3.12236492 3.42100001
 3.02529108 2.07944154 3.31054301 2.99071973 3.08648664 3.37073817
 3.59456877 2.41591378 3.2308044  3.78645978 2.92316158 2.79728133
 2.84490938 3.33576958 3.10906096 3.11351531

In [ ]:
import tensorflow_io as tfio



try:
    xrange  # Python 2
except NameError:
    xrange = range  # Python 3

def _make_input_parser(with_target=True):
  """Returns a parser func according to file_type, task_type and target.
  Need to set record_default for last column to integer instead of float in
  case of classification tasks.
  Args:
    with_target (boolean): Pass label or not.
  Returns:
    It returns a parser.
  """

  def _decode_csv(line):
    """Takes the string input tensor and parses it to feature dict and target.
    All the columns except the first one are treated as feature column. The
    first column is expected to be the target.
    Only returns target for if with_target is True.
    Args:
      line: csv rows in tensor format.
    Returns:
      features: A dictionary of features with key as "column_names" from
        self._column_header.
      target: tensor of target values which is the first column of the file.
        This will only be returned if with_target==True.
    """
#     column_header = feat_nam_lst if with_target else feat_nam_lst[:4]
    column_header = feat_nam_lst
    record_defaults = [[0.] for _ in xrange(len(feat_nam_lst))]
    # Pass label as integer.
    columns = tf.io.decode_csv(line, record_defaults=record_defaults)
#     columns = tfio.experimental.serialization.decode_json(line, specs)
    features = dict(zip(column_header, columns))
    target = None # features.pop(column_names[4]) if with_target else None
    return features, target

  return _decode_csv



def json_input_receiver_fn():
    inputs = {}
    for feat in feature_columns:
        inputs[feat.name] = tf.compat.v1.placeholder(shape=[None], dtype=feat.dtype)
    csv_row = tf.compat.v1.placeholder(shape=[None], dtype=tf.string)
    features, _ = _make_input_parser(with_target=False)(csv_row)    
    return tf.estimator.export.ServingInputReceiver(features, {'inputs': csv_row}, {'json': features})


def _serving_input_fn():
    """Builds the serving inputs.

    Returns:
      A ServingInputReceiver with the placeholders and original features.
    """
    raw_features = {}
    converted_features = {}
    for feature_name in feat_nam_lst: #['CRIM','ZN','INDUS','CHAS','NOX','RM', 'AGE','DIS', 'RAD','TAX','PTRATIO','B', 'LSTAT']:
        raw_features[feature_name] = tf.compat.v1.placeholder(
          shape=[None], dtype=tf.float32)
        converted_features[feature_name] = raw_features[feature_name]

#     for feature_name in self.categorical_features:
#       if feature_name in self.boolean_type_features:
#         raw_features[feature_name] = tf.placeholder(shape=[None], dtype=tf.bool)
#         converted_features[feature_name] = tf.as_string(
#             raw_features[feature_name])
#       else:
#         raw_features[feature_name] = tf.placeholder(
#             shape=[None], dtype=tf.string)
#         converted_features[feature_name] = raw_features[feature_name]

#     for feature_name in self.array_features:
#       if feature_name in self.boolean_type_features:
#         raw_features[feature_name] = tf.placeholder(shape=[None], dtype=tf.bool)
#         converted_features[feature_name] = tf.as_string(
#             raw_features[feature_name])
#       else:
#         raw_features[feature_name] = tf.placeholder(
#             shape=[None], dtype=tf.string)
#         converted_features[feature_name] = raw_features[feature_name]

    return tf.estimator.export.ServingInputReceiver(converted_features,
                                                    raw_features)


estimator.export_saved_model('./adanet3', _serving_input_fn)


In [21]:
!saved_model_cli show --dir ./adanet3/1591753720 --all

2020-09-10 14:09:04.071024: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['AGE'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_6:0
    inputs['B'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_11:0
    inputs['CHAS'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_3:0
    inputs['CRIM'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder:0
    inputs['DIS'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_7:0
    inputs['INDUS'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_2:0
    inputs['LSTAT'] tensor_info:
        dtype: DT_FL

## Conclusion and next steps

In this tutorial, you learned how to customize `adanet` to encode your
understanding of a particular dataset, and explore novel search spaces with
AdaNet.

One use-case that has worked for us at Google, has been to take a production
model's TensorFlow code, convert it to into an `adanet.subnetwork.Builder`, and
adaptively grow it into an ensemble. In many cases, this has given significant
performance improvements.

As an exercise, you can swap out the FASHION-MNIST with the MNIST handwritten
digits dataset in this notebook using `tf.keras.datasets.mnist.load_data()`, and
see how `SimpleCNN` performs.

In [ ]:
# imported.signatures["predict"]({'feat0': tf.constant(18.0846),   
#                                 'feat1': tf.constant(0.),  
#                                 'feat2': tf.constant(18.1),   
#                                 'feat3': tf.constant(0.),   
#                                 'feat4': tf.constant(0.679),   
#                                 'feat5': tf.constant(6.434),
#                                 'feat6': tf.constant(100.),   
#                                 'feat7': tf.constant(1.8347),  
#                                 'feat8': tf.constant(24.), 
#                                 'feat8': tf.constant(666.),  
#                                 'feat9': tf.constant(20.2),  
#                                 'feat10': tf.constant(27.25),
#                                 'feat11': tf.constant(29.05),
#                                 'feat12': tf.constant(12.0)})

imported = tf.compat.v2.saved_model.load("./adanet3/1591741163")
infer = imported.signatures["predict"]


# infer( feat0=tf.constant(18.0846), feat1=tf.constant(0.),feat2=tf.constant(18.1), feat3=tf.constant(0.), 
#         feat4=tf.constant(0.679), feat5=tf.constant(6.434),feat6=tf.constant(100.),
#         feat7=tf.constant(1.8347),feat8=tf.constant(24.),feat9=tf.constant(666.),
#         feat10=tf.constant(20.2), feat11=tf.constant(27.25),feat12=tf.constant(29.05))

In [ ]:
!gsutil cp -r ./adanet3/1591753720 gs://${BUCKET}/boston/adanet/

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
boston = load_boston()
features = np.array(pd.Series(boston['feature_names'], dtype=pd.StringDtype()))
features = np.concatenate([features, ['target']]) 
data =  pd.DataFrame(data= np.c_[boston['data'], boston['target']],
                     columns= features )